<a href="https://colab.research.google.com/github/yoshi-p27/Aayush-Patel-Assessment-Task/blob/main/Aayush_Patel_Assessment_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📝 Markdown to Google Docs Converter

This notebook converts markdown meeting notes into a well-formatted Google Document.

**Features:**
- ✅ Heading hierarchy (H1, H2, H3)
- ✅ Nested bullet points
- ✅ Checkboxes for action items
- ✅ @mention styling
- ✅ Footer formatting

**Time to run:** ~30 seconds

**Output:** A formatted Google Doc with a shareable link

In [1]:
# Import required libraries
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import re

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


In [2]:
# Markdown meeting notes to convert
MEETING_NOTES = """# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes"""

print("✓ Meeting notes loaded")
print(f"📄 Content length: {len(MEETING_NOTES)} characters")

✓ Meeting notes loaded
📄 Content length: 1332 characters


In [3]:
class MarkdownToGoogleDocs:
    """Converts markdown text to a formatted Google Doc."""

    def __init__(self):
        """Initialize the converter and authenticate."""
        self.service = None
        self.doc_id = None
        self.requests = []
        self.current_index = 1  # Start after title

    def authenticate(self):
        """Authenticate with Google API in Colab environment."""
        try:
            auth.authenticate_user()
            self.service = build('docs', 'v1')
            print("✓ Authentication successful")
        except Exception as e:
            raise Exception(f"Authentication failed: {str(e)}")

    def create_document(self, title="Product Team Sync - May 15, 2023"):
        """Create a new Google Doc."""
        try:
            doc = self.service.documents().create(body={'title': title}).execute()
            self.doc_id = doc.get('documentId')
            print(f"✓ Document created: https://docs.google.com/document/d/{self.doc_id}/edit")
            return self.doc_id
        except HttpError as e:
            raise Exception(f"Failed to create document: {str(e)}")

    def parse_and_convert(self, markdown_text):
        """Parse markdown and generate Google Docs API requests."""
        lines = markdown_text.split('\n')
        i = 0

        while i < len(lines):
            line = lines[i]

            # Skip empty lines
            if not line.strip():
                i += 1
                continue

            # Handle H1 (Main Title)
            if line.startswith('# '):
                self._add_heading(line[2:], 'HEADING_1')

            # Handle H2 (Section Headers)
            elif line.startswith('## '):
                self._add_heading(line[3:], 'HEADING_2')

            # Handle H3 (Sub-section Headers)
            elif line.startswith('### '):
                self._add_heading(line[4:], 'HEADING_3')

            # Handle horizontal rule (footer separator)
            elif line.startswith('---'):
                self._add_text('\n')

            # Handle action items with checkboxes
            elif line.strip().startswith('- [ ]'):
                self._add_checkbox_item(line)

            # Handle bullet points
            elif line.strip().startswith(('* ', '- ')):
                indent_level = self._get_indent_level(line)
                text = line.strip()[2:]  # Remove bullet marker
                self._add_bullet_point(text, indent_level)

            # Handle regular text
            else:
                self._add_text(line + '\n')

            i += 1

    def _get_indent_level(self, line):
        """Calculate indentation level based on leading spaces."""
        spaces = len(line) - len(line.lstrip())
        return spaces // 2

    def _add_heading(self, text, style):
        """Add a heading with specified style."""
        self.requests.append({
            'insertText': {
                'location': {'index': self.current_index},
                'text': text + '\n'
            }
        })

        end_index = self.current_index + len(text)

        self.requests.append({
            'updateParagraphStyle': {
                'range': {
                    'startIndex': self.current_index,
                    'endIndex': end_index + 1
                },
                'paragraphStyle': {
                    'namedStyleType': style
                },
                'fields': 'namedStyleType'
            }
        })

        self.current_index = end_index + 1

    def _add_bullet_point(self, text, indent_level):
        """Add a bullet point with proper indentation."""
        # Check for @mentions and format them
        formatted_text, mention_ranges = self._format_mentions(text)

        self.requests.append({
            'insertText': {
                'location': {'index': self.current_index},
                'text': formatted_text + '\n'
            }
        })

        start_index = self.current_index
        end_index = self.current_index + len(formatted_text) + 1

        # Apply bullet formatting
        self.requests.append({
            'createParagraphBullets': {
                'range': {
                    'startIndex': start_index,
                    'endIndex': end_index
                },
                'bulletPreset': 'BULLET_DISC_CIRCLE_SQUARE'
            }
        })

        # Apply indentation
        if indent_level > 0:
            self.requests.append({
                'updateParagraphStyle': {
                    'range': {
                        'startIndex': start_index,
                        'endIndex': end_index
                    },
                    'paragraphStyle': {
                        'indentStart': {
                            'magnitude': 36 * indent_level,
                            'unit': 'PT'
                        }
                    },
                    'fields': 'indentStart'
                }
            })

        # Format @mentions
        for mention_start, mention_end in mention_ranges:
            abs_start = start_index + mention_start
            abs_end = start_index + mention_end
            self.requests.append({
                'updateTextStyle': {
                    'range': {
                        'startIndex': abs_start,
                        'endIndex': abs_end
                    },
                    'textStyle': {
                        'bold': True,
                        'foregroundColor': {
                            'color': {
                                'rgbColor': {
                                    'red': 0.2,
                                    'green': 0.4,
                                    'blue': 0.8
                                }
                            }
                        }
                    },
                    'fields': 'bold,foregroundColor'
                }
            })

        self.current_index = end_index

    def _add_checkbox_item(self, line):
        """Add a checkbox item for action items."""
        # Extract text after "- [ ]"
        text = line.strip()[6:]  # Remove "- [ ] "
        formatted_text, mention_ranges = self._format_mentions(text)

        self.requests.append({
            'insertText': {
                'location': {'index': self.current_index},
                'text': formatted_text + '\n'
            }
        })

        start_index = self.current_index
        end_index = self.current_index + len(formatted_text) + 1

        # Create checkbox bullet
        self.requests.append({
            'createParagraphBullets': {
                'range': {
                    'startIndex': start_index,
                    'endIndex': end_index
                },
                'bulletPreset': 'BULLET_CHECKBOX'
            }
        })

        # Format @mentions
        for mention_start, mention_end in mention_ranges:
            abs_start = start_index + mention_start
            abs_end = start_index + mention_end
            self.requests.append({
                'updateTextStyle': {
                    'range': {
                        'startIndex': abs_start,
                        'endIndex': abs_end
                    },
                    'textStyle': {
                        'bold': True,
                        'foregroundColor': {
                            'color': {
                                'rgbColor': {
                                    'red': 0.2,
                                    'green': 0.4,
                                    'blue': 0.8
                                }
                            }
                        }
                    },
                    'fields': 'bold,foregroundColor'
                }
            })

        self.current_index = end_index

    def _format_mentions(self, text):
        """Extract @mentions and return formatted text with mention positions."""
        mention_pattern = r'@\w+'
        mentions = []

        for match in re.finditer(mention_pattern, text):
            mentions.append((match.start(), match.end()))

        return text, mentions

    def _add_text(self, text):
        """Add regular text."""
        # Check for footer information
        if text.startswith('Meeting recorded by:') or text.startswith('Duration:'):
            self.requests.append({
                'insertText': {
                    'location': {'index': self.current_index},
                    'text': text
                }
            })

            end_index = self.current_index + len(text)

            # Style footer text
            self.requests.append({
                'updateTextStyle': {
                    'range': {
                        'startIndex': self.current_index,
                        'endIndex': end_index
                    },
                    'textStyle': {
                        'italic': True,
                        'fontSize': {
                            'magnitude': 10,
                            'unit': 'PT'
                        },
                        'foregroundColor': {
                            'color': {
                                'rgbColor': {
                                    'red': 0.5,
                                    'green': 0.5,
                                    'blue': 0.5
                                }
                            }
                        }
                    },
                    'fields': 'italic,fontSize,foregroundColor'
                }
            })

            self.current_index = end_index
        else:
            self.requests.append({
                'insertText': {
                    'location': {'index': self.current_index},
                    'text': text
                }
            })
            self.current_index += len(text)

    def apply_formatting(self):
        """Apply all formatting requests to the document."""
        try:
            if self.requests:
                self.service.documents().batchUpdate(
                    documentId=self.doc_id,
                    body={'requests': self.requests}
                ).execute()
                print("✓ Formatting applied successfully")
        except HttpError as e:
            raise Exception(f"Failed to apply formatting: {str(e)}")

    def convert(self, markdown_text):
        """Main conversion method."""
        try:
            print("Starting conversion process...")
            self.authenticate()
            self.create_document()
            self.parse_and_convert(markdown_text)
            self.apply_formatting()
            print(f"\n✓ Conversion complete!")
            print(f"📄 View document: https://docs.google.com/document/d/{self.doc_id}/edit")
            return self.doc_id
        except Exception as e:
            print(f"✗ Error during conversion: {str(e)}")
            raise

print("✓ MarkdownToGoogleDocs class defined")

✓ MarkdownToGoogleDocs class defined


In [4]:
# Create converter instance and run conversion
converter = MarkdownToGoogleDocs()
doc_id = converter.convert(MEETING_NOTES)

Starting conversion process...
✓ Authentication successful
✓ Document created: https://docs.google.com/document/d/1lNtnVTMAfdgBQidLilHjcdddflFVa5yHtD4jr8nxu7A/edit
✓ Formatting applied successfully

✓ Conversion complete!
📄 View document: https://docs.google.com/document/d/1lNtnVTMAfdgBQidLilHjcdddflFVa5yHtD4jr8nxu7A/edit


## 📖 How to Use This Notebook

1. **Run Cell 2** - Import dependencies
2. **Run Cell 3** - Load meeting notes
3. **Run Cell 4** - Define the converter class
4. **Run Cell 5** - Execute the conversion
   - You'll be prompted to authenticate
   - Click the link and authorize access
   - The document will be created automatically

5. **Click the output link** to view your formatted Google Doc!

---

## 🔧 Troubleshooting

**Authentication Issues:**
- Make sure you're logged into a Google account
- Grant all requested permissions
- If it fails, restart runtime and try again

**API Errors:**
- Check your internet connection
- Ensure Google Docs API is not blocked

---

## 📝 Notes

- Each run creates a **new** Google Doc
- The document is saved to your Google Drive
- You can modify `MEETING_NOTES` in Cell 3 for different content
- All formatting is applied automatically